# Reduced Model with autoReduce attempt 11
Try some models for presentation at lab meeting <br>
7.13.2020 <br>


In [1]:
# imports
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import tqdm
import math

Loading BokehJS ...

**What models do I want to show?**
- Remove all complexes (desired)
- Remove every other complex (abridged desired)
- Remove all enzymes (maybe desired)
- Remove only one complex (control)
- Remove everything except atp, glucose, isobutanol (ultra desired)

**What species do I want to plot?**
- glucose
- atp
- isobutanol


In [2]:
%run prep_for_autoreduce.py

In [3]:
# Extract data for glucose, atp, isobutanol
x_init

array([ 0.15, 30.  , 30.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.15,  0.  ,
        0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,
        0.  ,  0.15, 30.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.15, 30.  ,
        0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,
        0.  ,  0.15,  0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.  ,
        0.15,  0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.  ,  0.15,
        0.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.  ,  0.  ,  0.15,  0.  ,
        0.  ,  0.  ,  2.  ,  0.  ,  0.  ])

In [4]:
p_reg = bokeh.plotting.figure(width = 475, height = 350, title = 'Full Model from SBML',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p_reg.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p_reg.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p_reg.line(timepoints_ode, sol[:,62], line_width = 2,legend_label = 'isobutanol', color = colors[1])
bokeh.io.show(p_reg)

**Make list of reduced models**

In [5]:
remove_list = [ [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,66,67], # all complexes
               [5,9,13,17,23,28,31,35,39,43,47,51,55,59,63,66], # every other complex
               [0,7,11,15,19,25,30,33, 37, 41, 45, 49, 53, 57, 61, 65], # all enzymes
               [5] # one complex
               ]
remove_everything = np.arange(0,68)

In [6]:
remove_everything_list = list(remove_everything)

In [7]:
remove_everything_list.remove(1)
remove_everything_list.remove(2)
remove_everything_list.remove(62)

### Try entire process with plotting on remove_list[0],all complexes

In [8]:
curr_list = remove_list[0]
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)      

In [9]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

In [10]:
aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/sympy/core/relational.py:470: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  deprecated_since_version="1.5"


Successful time-scale separation solution obtained.


108.4455089427204

In [11]:
# Define correct initial conditions
spec_inds = [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65] # indices of stuff we decided to keep
# Setup initial conditions

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 5, 7,9, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(spec_inds):
    if item in enz_list:
        x_init[i] = 0.15
    elif item in mol_list:
        x_init[i] = 30
    else:
        x_init[i] = 0
        

# for i,item in enumerate(x_init):
#     if i in enz_list and i in spec_inds:
#         x_init[i] = 0.15
        
#     # atpase init cond
#     if i == 65 and i in spec_inds:
#         x_init[i] = 2.
        
#     # set up other molecules
#     if i in mol_list and i in spec_inds:
#         x_init[i] = 30


In [12]:
x_init[23]

0.15

In [13]:
from numpy import *
from matplotlib.pylab import *
from matplotlib.pyplot import *
from scipy.integrate import odeint 

def simulateModel(t0, tend, numPoints):
  
  #compartments
  default = 1e-06

  
  #boundary species
  
  def ode_fun(x, t): # REMOVE 
    x0 = enzyme_hex = x[0]
    x1 = metabolite_atp = x[1]
    x2 = molecule_glucose = x[2]
    x3 = molecule_g6p = x[3]
    x4 = metabolite_adp = x[4]
    x7 = enzyme_pgi = x[5]
    x8 = molecule_f6p = x[6]
    x11 = enzyme_pfk = x[7]
    x12 = molecule_f16p = x[8]
    x15 = enzyme_ald_tpi = x[9]
    x16 = molecule_g3p = x[10]
    x19 = enzyme_gapN = x[11]
    x20 = metabolite_nadp = x[12]
    x21 = molecule_3pg = x[13]
    x22 = metabolite_nadph = x[14]
    x25 = enzyme_mGapDH = x[15]
    x26 = metabolite_pi = x[16]
    x27 = molecule_13bpg = x[17]
    x30 = enzyme_pgk = x[18]
    x33 = enzyme_pgm = x[19]
    x34 = molecule_2pg = x[20]
    x37 = enzyme_eno = x[21]
    x38 = molecule_pep = x[22]
    x41 = enzyme_pyk = x[23]
    x42 = molecule_pyruvate = x[24]
    x45 = enzyme_alsS = x[25]
    x46 = molecule_acetolac = x[26]
    x49 = enzyme_IlvC = x[27]
    x50 = molecule_23dih3mebut = x[28]
    x53 = enzyme_IlvD = x[29]
    x54 = molecule_3me2oxo = x[30]
    x57 = enzyme_kivD = x[31]
    x58 = molecule_isobutanal = x[32]
    x61 = enzyme_yahk = x[33]
    x62 = molecule_isobutanol = x[34]
    x65 = enzyme_atpase = x[35]


    return array([-14.9993750260406*x0*x1*x2 - 1.5*x0*x3*x4 + 2.0e-13*x0*(74996875130203.0*x1*x2 + 7500000000000.0*x3*x4),
 -14.9993750260406*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 14.9993750260406*x1*x11*x8 - 1.5*x1*x21**2*x30 - 8.57142857142857*x1*x65 + 2.0e-13*x30*(7500000000000.0*x1*x21**2 + 74996875130203.0*x27*x4) + 4.0e-13*x41*(7500000000000.0*x1**2*x42**2 + 74996875130203.0*x38**2*x4**2),
 -14.9993750260406*x0*x1*x2,
 -1.5*x0*x3*x4 + 2.0e-13*x0*(74996875130203.0*x1*x2 + 7500000000000.0*x3*x4) - 14.9993750260406*x3*x7,
 -1.5*x0*x3*x4 + 2.0e-13*x0*(74996875130203.0*x1*x2 + 7500000000000.0*x3*x4) - 1.5*x11*x12*x4 + 2.0e-13*x11*(74996875130203.0*x1*x8 + 7500000000000.0*x12*x4) - 1.5*x26*x4*x65 - 14.9993750260406*x27*x30*x4 - 29.9987500520812*x38**2*x4**2*x41 + 0.214285714285714*x65*(40.0*x1 + 7.0*x26*x4),
 -14.9993750260406*x3*x7 - 1.5*x7*x8 + 2.0e-13*x7*(74996875130203.0*x3 + 7500000000000.0*x8),
 -14.9993750260406*x1*x11*x8 - 1.5*x7*x8 + 2.0e-13*x7*(74996875130203.0*x3 + 7500000000000.0*x8),
 -14.9993750260406*x1*x11*x8 - 1.5*x11*x12*x4 + 2.0e-13*x11*(74996875130203.0*x1*x8 + 7500000000000.0*x12*x4),
 -1.5*x11*x12*x4 + 2.0e-13*x11*(74996875130203.0*x1*x8 + 7500000000000.0*x12*x4) - 14.9993750260406*x12*x15,
 -14.9993750260406*x12*x15 - 1.5*x15*x16**2 + 2.0e-13*x15*(74996875130203.0*x12 + 7500000000000.0*x16**2),
 -3.0*x15*x16**2 + 4.0e-13*x15*(74996875130203.0*x12 + 7500000000000.0*x16**2) - 29.9987500520812*x16**2*x19*x20**2 - 29.9987500520812*x16**2*x20**2*x25*x26,
 -14.9993750260406*x16**2*x19*x20**2 - 1.5*x19*x21**2*x22**2 + 2.0e-13*x19*(74996875130203.0*x16**2*x20**2 + 7500000000000.0*x21**2*x22**2),
 -29.9987500520812*x16**2*x19*x20**2 - 29.9987500520812*x16**2*x20**2*x25*x26 - 1.5*x20*x49*x50 - 1.5*x20*x61*x62 + 2.0e-13*x49*(7500000000000.0*x20*x50 + 74996875130203.0*x22*x46) + 2.0e-13*x61*(7500000000000.0*x20*x62 + 74996875130203.0*x22*x58),
 -3.0*x1*x21**2*x30 - 3.0*x19*x21**2*x22**2 + 4.0e-13*x19*(74996875130203.0*x16**2*x20**2 + 7500000000000.0*x21**2*x22**2) - 29.9987500520812*x21**2*x33 + 4.0e-13*x30*(7500000000000.0*x1*x21**2 + 74996875130203.0*x27*x4),
 -3.0*x19*x21**2*x22**2 + 4.0e-13*x19*(74996875130203.0*x16**2*x20**2 + 7500000000000.0*x21**2*x22**2) - 3.0*x22**2*x25*x27 - 14.9993750260406*x22*x46*x49 - 14.9993750260406*x22*x58*x61 + 4.0e-13*x25*(74996875130203.0*x16**2*x20**2*x26 + 7500000000000.0*x22**2*x27),
 -14.9993750260406*x16**2*x20**2*x25*x26 - 1.5*x22**2*x25*x27 + 2.0e-13*x25*(74996875130203.0*x16**2*x20**2*x26 + 7500000000000.0*x22**2*x27),
 -14.9993750260406*x16**2*x20**2*x25*x26 - 1.5*x26*x4*x65 + 0.214285714285714*x65*(40.0*x1 + 7.0*x26*x4),
 -1.5*x22**2*x25*x27 + 2.0e-13*x25*(74996875130203.0*x16**2*x20**2*x26 + 7500000000000.0*x22**2*x27) - 14.9993750260406*x27*x30*x4,
 -1.5*x1*x21**2*x30 - 14.9993750260406*x27*x30*x4 + 2.0e-13*x30*(7500000000000.0*x1*x21**2 + 74996875130203.0*x27*x4),
 -14.9993750260406*x21**2*x33 - 1.5*x33*x34**2 + 2.0e-13*x33*(74996875130203.0*x21**2 + 7500000000000.0*x34**2),
 -3.0*x33*x34**2 + 4.0e-13*x33*(74996875130203.0*x21**2 + 7500000000000.0*x34**2) - 29.9987500520812*x34**2*x37,
 -14.9993750260406*x34**2*x37 - 1.5*x37*x38**2 + 2.0e-13*x37*(74996875130203.0*x34**2 + 7500000000000.0*x38**2),
 -3.0*x37*x38**2 + 4.0e-13*x37*(74996875130203.0*x34**2 + 7500000000000.0*x38**2) - 29.9987500520812*x38**2*x4**2*x41,
 -1.5*x1**2*x41*x42**2 - 14.9993750260406*x38**2*x4**2*x41 + 2.0e-13*x41*(7500000000000.0*x1**2*x42**2 + 74996875130203.0*x38**2*x4**2),
 -3.0*x1**2*x41*x42**2 + 4.0e-13*x41*(7500000000000.0*x1**2*x42**2 + 74996875130203.0*x38**2*x4**2) - 29.9987500520812*x42**2*x45,
 -14.9993750260406*x42**2*x45 - 1.5*x45*x46 + 2.0e-13*x45*(74996875130203.0*x42**2 + 7500000000000.0*x46),
 -14.9993750260406*x22*x46*x49 - 1.5*x45*x46 + 2.0e-13*x45*(74996875130203.0*x42**2 + 7500000000000.0*x46),
 -1.5*x20*x49*x50 - 14.9993750260406*x22*x46*x49 + 2.0e-13*x49*(7500000000000.0*x20*x50 + 74996875130203.0*x22*x46),
 -1.5*x20*x49*x50 + 2.0e-13*x49*(7500000000000.0*x20*x50 + 74996875130203.0*x22*x46) - 14.9993750260406*x50*x53,
 -14.9993750260406*x50*x53 - 1.5*x53*x54 + 2.0e-13*x53*(74996875130203.0*x50 + 7500000000000.0*x54),
 -1.5*x53*x54 + 2.0e-13*x53*(74996875130203.0*x50 + 7500000000000.0*x54) - 14.9993750260406*x54*x57,
 -14.9993750260406*x54*x57 - 1.5*x57*x58 + 2.0e-13*x57*(74996875130203.0*x54 + 7500000000000.0*x58),
 -14.9993750260406*x22*x58*x61 - 1.5*x57*x58 + 2.0e-13*x57*(74996875130203.0*x54 + 7500000000000.0*x58),
 -1.5*x20*x61*x62 - 14.9993750260406*x22*x58*x61 + 2.0e-13*x61*(7500000000000.0*x20*x62 + 74996875130203.0*x22*x58),
 -1.5*x20*x61*x62 + 2.0e-13*x61*(7500000000000.0*x20*x62 + 74996875130203.0*x22*x58),
 -8.57142857142857*x1*x65 - 1.5*x26*x4*x65 + 0.214285714285714*x65*(40.0*x1 + 7.0*x26*x4)])

  time = linspace(t0, tend, numPoints)
  #yinit= array([0.15, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0])
  yinit = x_init
  # Define correct yinit

  y = odeint(ode_fun, yinit, time)

  return time, y


time, result = simulateModel(0, 75, 1000)

p2 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 1: Remove All Complexes',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p2.line(time, result[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p2.line(time, result[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p2.line(time, result[:,34], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p2.legend.click_policy = 'hide'
bokeh.io.show(p2)




## Entire process for every other complex

In [14]:
%run prep_for_autoreduce.py

In [15]:
curr_list = remove_list[1]
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

attempting to retain : [0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 32, 33, 34, 36, 37, 38, 40, 41, 42, 44, 45, 46, 48, 49, 50, 52, 53, 54, 56, 57, 58, 60, 61, 62, 64, 65, 67]
Successful time-scale separation solution obtained.


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


236.17683197778143

In [16]:
aa.f

[-14.9993750260406*x0*x1*x2 - 1.5*x0*x3*x4 + 15.0*x6,
 -14.9993750260406*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 14.9993750260406*x1*x11*x8 - 1.5*x1*x21**2*x30 - 8.57142857142857*x1*x65 + 15.0*x32 + 30.0*x44,
 -14.9993750260406*x0*x1*x2,
 -1.5*x0*x3*x4 - 14.9993750260406*x3*x7 + 15.0*x6,
 -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 14.9993750260406*x27*x30*x4 - 29.9987500520812*x38**2*x4**2*x41 + 15.0*x6 + 15.0*x67,
 14.9993750260406*x0*x1*x2 + 1.5*x0*x3*x4 - 15.0*x6,
 15.0*x10 - 14.9993750260406*x3*x7 - 1.5*x7*x8,
 -14.9993750260406*x1*x11*x8 + 15.0*x10 - 1.5*x7*x8,
 -15.0*x10 + 14.9993750260406*x3*x7 + 1.5*x7*x8,
 -14.9993750260406*x1*x11*x8 - 1.5*x11*x12*x4 + 15.0*x14,
 -1.5*x11*x12*x4 - 14.9993750260406*x12*x15 + 15.0*x14,
 14.9993750260406*x1*x11*x8 + 1.5*x11*x12*x4 - 15.0*x14,
 -14.9993750260406*x12*x15 - 1.5*x15*x16**2 + 15.0*x18,
 -3.0*x15*x16**2 - 29.9987500520812*x16**2*x19*x20**2 - 29.9987500520812*x16**2*x20**2*x25*x26 + 30.0*x18,
 14.9993750260406*x12*x15 + 1.5*

In [17]:

# Define correct initial conditions
spec_inds = [0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 32, 
             33, 34, 36, 37, 38, 40, 41, 42, 44, 45, 46, 48, 49, 50, 52, 53, 54, 56, 57, 58, 60, 61, 62, 64, 65, 67] # indices of stuff we decided to keep
# Setup initial conditions

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 5, 7,9, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(spec_inds):
    if item in enz_list:
        x_init[i] = 0.20
    elif item in mol_list:
        x_init[i] = 30
    else:
        x_init[i] = 0
        



In [18]:
len(x_init)

52

In [19]:
from numpy import *
from matplotlib.pylab import *
from matplotlib.pyplot import *
from scipy.integrate import odeint 

def simulateModel(t0, tend, numPoints):
  
  #compartments
  default = 1e-06

  
  #boundary species
  
  def ode_fun(x, t): # REMOVE 
    
    x0 = enzyme_hex = x[0]
    x1 = metabolite_atp = x[1]
    x2 = molecule_glucose = x[2]
    x3 = molecule_g6p = x[3]
    x4 = metabolite_adp = x[4]
    x6 = complex_enzyme_hex_metabolite_adp_molecule_g6p = x[5]
    x7 = enzyme_pgi = x[6]
    x8 = molecule_f6p = x[7]
    x10 = complex_enzyme_pgi_molecule_f6p = x[8]
    x11 = enzyme_pfk = x[9]
    x12 = molecule_f16p = x[10]
    x14 = complex_enzyme_pfk_metabolite_adp_molecule_f16p = x[11]
    x15 = enzyme_ald_tpi = x[12]
    x16 = molecule_g3p = x[13]
    x18 = complex_enzyme_ald_tpi_2x_molecule_g3p = x[14]
    x19 = enzyme_gapN = x[15]
    x20 = metabolite_nadp = x[16]
    x21 = molecule_3pg = x[17]
    x22 = metabolite_nadph = x[18]
    x24 = complex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg = x[19]
    x25 = enzyme_mGapDH = x[20]
    x26 = metabolite_pi = x[21]
    x27 = molecule_13bpg = x[22]
    x29 = complex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg = x[23]
    x30 = enzyme_pgk = x[24]
    x32 = complex_enzyme_pgk_metabolite_atp_2x_molecule_3pg = x[25]
    x33 = enzyme_pgm = x[26]
    x34 = molecule_2pg = x[27]
    x36 = complex_enzyme_pgm_2x_molecule_2pg = x[28]
    x37 = enzyme_eno = x[29]
    x38 = molecule_pep = x[30]
    x40 = complex_enzyme_eno_2x_molecule_pep = x[31]
    x41 = enzyme_pyk = x[32]
    x42 = molecule_pyruvate = x[33]
    x44 = complex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate = x[34]
    x45 = enzyme_alsS = x[35]
    x46 = molecule_acetolac = x[36]
    x48 = complex_enzyme_alsS_molecule_acetolac = x[37]
    x49 = enzyme_IlvC = x[38]
    x50 = molecule_23dih3mebut = x[39]
    x52 = complex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut = x[40]
    x53 = enzyme_IlvD = x[41]
    x54 = molecule_3me2oxo = x[42]
    x56 = complex_enzyme_IlvD_molecule_3me2oxo = x[43]
    x57 = enzyme_kivD = x[44]
    x58 = molecule_isobutanal = x[45]
    x60 = complex_enzyme_kivD_molecule_isobutanal = x[46]
    x61 = enzyme_yahk = x[47]
    x62 = molecule_isobutanol = x[48]
    x64 = complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol = x[49]
    x65 = enzyme_atpase = x[50]
    x67 = complex_enzyme_atpase_metabolite_adp_metabolite_pi = x[51]


    return array([-14.9993750260406*x0*x1*x2 - 1.5*x0*x3*x4 + 15.0*x6,
 -14.9993750260406*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 14.9993750260406*x1*x11*x8 - 1.5*x1*x21**2*x30 - 8.57142857142857*x1*x65 + 15.0*x32 + 30.0*x44,
 -14.9993750260406*x0*x1*x2,
 -1.5*x0*x3*x4 - 14.9993750260406*x3*x7 + 15.0*x6,
 -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 14.9993750260406*x27*x30*x4 - 29.9987500520812*x38**2*x4**2*x41 + 15.0*x6 + 15.0*x67,
 14.9993750260406*x0*x1*x2 + 1.5*x0*x3*x4 - 15.0*x6,
 15.0*x10 - 14.9993750260406*x3*x7 - 1.5*x7*x8,
 -14.9993750260406*x1*x11*x8 + 15.0*x10 - 1.5*x7*x8,
 -15.0*x10 + 14.9993750260406*x3*x7 + 1.5*x7*x8,
 -14.9993750260406*x1*x11*x8 - 1.5*x11*x12*x4 + 15.0*x14,
 -1.5*x11*x12*x4 - 14.9993750260406*x12*x15 + 15.0*x14,
 14.9993750260406*x1*x11*x8 + 1.5*x11*x12*x4 - 15.0*x14,
 -14.9993750260406*x12*x15 - 1.5*x15*x16**2 + 15.0*x18,
 -3.0*x15*x16**2 - 29.9987500520812*x16**2*x19*x20**2 - 29.9987500520812*x16**2*x20**2*x25*x26 + 30.0*x18,
 14.9993750260406*x12*x15 + 1.5*x15*x16**2 - 15.0*x18,
 -14.9993750260406*x16**2*x19*x20**2 - 1.5*x19*x21**2*x22**2 + 15.0*x24,
 -29.9987500520812*x16**2*x19*x20**2 - 29.9987500520812*x16**2*x20**2*x25*x26 - 1.5*x20*x49*x50 - 1.5*x20*x61*x62 + 15.0*x52 + 15.0*x64,
 -3.0*x1*x21**2*x30 - 3.0*x19*x21**2*x22**2 - 29.9987500520812*x21**2*x33 + 30.0*x24 + 30.0*x32,
 -3.0*x19*x21**2*x22**2 - 3.0*x22**2*x25*x27 - 14.9993750260406*x22*x46*x49 - 14.9993750260406*x22*x58*x61 + 30.0*x24 + 30.0*x29,
 14.9993750260406*x16**2*x19*x20**2 + 1.5*x19*x21**2*x22**2 - 15.0*x24,
 -14.9993750260406*x16**2*x20**2*x25*x26 - 1.5*x22**2*x25*x27 + 15.0*x29,
 -14.9993750260406*x16**2*x20**2*x25*x26 - 1.5*x26*x4*x65 + 15.0*x67,
 -1.5*x22**2*x25*x27 - 14.9993750260406*x27*x30*x4 + 15.0*x29,
 14.9993750260406*x16**2*x20**2*x25*x26 + 1.5*x22**2*x25*x27 - 15.0*x29,
 -1.5*x1*x21**2*x30 - 14.9993750260406*x27*x30*x4 + 15.0*x32,
 1.5*x1*x21**2*x30 + 14.9993750260406*x27*x30*x4 - 15.0*x32,
 -14.9993750260406*x21**2*x33 - 1.5*x33*x34**2 + 15.0*x36,
 -3.0*x33*x34**2 - 29.9987500520812*x34**2*x37 + 30.0*x36,
 14.9993750260406*x21**2*x33 + 1.5*x33*x34**2 - 15.0*x36,
 -14.9993750260406*x34**2*x37 - 1.5*x37*x38**2 + 15.0*x40,
 -3.0*x37*x38**2 - 29.9987500520812*x38**2*x4**2*x41 + 30.0*x40,
 14.9993750260406*x34**2*x37 + 1.5*x37*x38**2 - 15.0*x40,
 -1.5*x1**2*x41*x42**2 - 14.9993750260406*x38**2*x4**2*x41 + 15.0*x44,
 -3.0*x1**2*x41*x42**2 - 29.9987500520812*x42**2*x45 + 30.0*x44,
 1.5*x1**2*x41*x42**2 + 14.9993750260406*x38**2*x4**2*x41 - 15.0*x44,
 -14.9993750260406*x42**2*x45 - 1.5*x45*x46 + 15.0*x48,
 -14.9993750260406*x22*x46*x49 - 1.5*x45*x46 + 15.0*x48,
 14.9993750260406*x42**2*x45 + 1.5*x45*x46 - 15.0*x48,
 -1.5*x20*x49*x50 - 14.9993750260406*x22*x46*x49 + 15.0*x52,
 -1.5*x20*x49*x50 - 14.9993750260406*x50*x53 + 15.0*x52,
 1.5*x20*x49*x50 + 14.9993750260406*x22*x46*x49 - 15.0*x52,
 -14.9993750260406*x50*x53 - 1.5*x53*x54 + 15.0*x56,
 -1.5*x53*x54 - 14.9993750260406*x54*x57 + 15.0*x56,
 14.9993750260406*x50*x53 + 1.5*x53*x54 - 15.0*x56,
 -14.9993750260406*x54*x57 - 1.5*x57*x58 + 15.0*x60,
 -14.9993750260406*x22*x58*x61 - 1.5*x57*x58 + 15.0*x60,
 14.9993750260406*x54*x57 + 1.5*x57*x58 - 15.0*x60,
 -1.5*x20*x61*x62 - 14.9993750260406*x22*x58*x61 + 15.0*x64,
 -1.5*x20*x61*x62 + 15.0*x64,
 1.5*x20*x61*x62 + 14.9993750260406*x22*x58*x61 - 15.0*x64,
 -8.57142857142857*x1*x65 - 1.5*x26*x4*x65 + 15.0*x67,
 8.57142857142857*x1*x65 + 1.5*x26*x4*x65 - 15.0*x67])

  time = linspace(t0, tend, numPoints)
  #yinit= array([0.15, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0])
  yinit = x_init
  # Define correct yinit

  y = odeint(ode_fun, yinit, time)

  return time, y


time, result = simulateModel(0, 75, 1000)

p3 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 2: Remove 1/2 Complexes',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p3.line(time, result[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p3.line(time, result[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p3.line(time, result[:,48], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p3.legend.click_policy = 'hide'
bokeh.io.show(p3)





## Entire process with removing all enzymes

In [20]:
%run prep_for_autoreduce.py

In [21]:
remove_list[2]

[0, 7, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]

In [22]:
%run prep_for_autoreduce.py
curr_list = remove_list[2]
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,100,10)
timepoints_ode = np.linspace(0, 100,10)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

attempting to retain : [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 42, 43, 44, 46, 47, 48, 50, 51, 52, 54, 55, 56, 58, 59, 60, 62, 63, 64, 66, 67]
Successful time-scale separation solution obtained.
The error is NaN, something wrong...continuing.


<string>:4: RuntimeWarning: invalid value encountered in double_scalars


nan

In [23]:
aa.f

[-3.0*x1**2*x42**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) - 15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 1.5*x1*x21**2*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 15.0*x1*x8*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) - 15.0*x1*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x5 + 1.5*x66,
 -15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 1.5*x5,
 -1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 15.0*x3*(10.0*x10 + x9)/(10.0*x3 + x8) + 15.0*x6 + 1.5*x9,
 -1.5*x12*x4*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) + 15.0*x14 - 1.5*x26*x4*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) - 15.0*x27*x4*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 1.5*x31 - 30.0*x38**2*x4**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) + 3.0*x43 + 15.0*x6 + 15.0*x67,
 15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 36001.5*x5,
 1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 36000.0*x5 - 15.0*x6,
 -15.0*x1*x8*(x13 + 10.0*x14)/(10.0*x1*

In [24]:
# Define correct initial conditions
spec_inds =  [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 42,
              43, 44, 46, 47, 48, 50, 51, 52, 54, 55, 56, 58, 59, 60, 62, 63, 64, 66, 67]

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 7, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61,65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(x_init):
    if i in enz_list and i in spec_inds:
        x_init[i] = 0.15
        
    # atpase init cond
    if i == 65 and i in spec_inds:
        x_init[i] = 2.
        
    # set up other molecules
    if i in mol_list and i in spec_inds:
        x_init[i] = 30




In [25]:
len(x_init)

53

In [26]:
from numpy import *
from matplotlib.pylab import *
from matplotlib.pyplot import *
from scipy.integrate import odeint 

def simulateModel(t0, tend, numPoints):
  
  #compartments
  default = 1e-06

  
  #boundary species
  
  def ode_fun(x, t): # REMOVE 
    x1 = metabolite_atp = x[0]
    x2 = molecule_glucose = x[1]
    x3 = molecule_g6p = x[2]
    x4 = metabolite_adp = x[3]
    x5 = complex_enzyme_hex_metabolite_atp_molecule_glucose = x[4]
    x6 = complex_enzyme_hex_metabolite_adp_molecule_g6p = x[5]
    x8 = molecule_f6p = x[6]
    x9 = complex_enzyme_pgi_molecule_g6p = x[7]
    x10 = complex_enzyme_pgi_molecule_f6p = x[8]
    x12 = molecule_f16p = x[9]
    x13= complex_enzyme_pfk_metabolite_atp_molecule_f6p = x[10]
    x14 = complex_enzyme_pfk_metabolite_adp_molecule_f16p = x[11]
    x16 = molecule_g3p = x[12]
    x17 = complex_enzyme_ald_tpi_molecule_f16p = x[13]
    x18 = complex_enzyme_ald_tpi_2x_molecule_g3p = x[14]
    x20 = metabolite_nadp = x[15]
    x21 = molecule_3pg = x[16]
    x22 = metabolite_nadph = x[17]
    x23 = complex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p = x[18]
    x24 = complex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg = x[19]
    x26 = metabolite_pi = x[20]
    x27 = molecule_13bpg = x[21]
    x28 = complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p = x[22]
    x29 = complex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg = x[23]
    x31 = complex_enzyme_pgk_metabolite_adp_molecule_13bpg = x[24]
    x32 = complex_enzyme_pgk_metabolite_atp_2x_molecule_3pg = x[25]
    x34 = molecule_2pg = x[26]
    x35 = complex_enzyme_pgm_2x_molecule_3pg = x[27]
    x36 = complex_enzyme_pgm_2x_molecule_2pg = x[28]
    x38 = molecule_pep = x[29]
    x39 = complex_enzyme_eno_2x_molecule_2pg = x[30]
    x40 = complex_enzyme_eno_2x_molecule_pep = x[31]
    x42 = molecule_pyruvate = x[32]
    x43 = complex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep = x[33]
    x44 = complex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate = x[34]
    x46 = molecule_acetolac = x[35]
    x47 = complex_enzyme_alsS_2x_molecule_pyruvate = x[36]
    x48 = complex_enzyme_alsS_molecule_acetolac = x[37]
    x50 = molecule_23dih3mebut = x[38]
    x51 = complex_enzyme_IlvC_metabolite_nadph_molecule_acetolac = x[39]
    x52 = complex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut = x[40]
    x54 = molecule_3me2oxo = x[41]
    x55 = complex_enzyme_IlvD_molecule_23dih3mebut = x[42]
    x56 = complex_enzyme_IlvD_molecule_3me2oxo = x[43]
    x58 = molecule_isobutanal = x[44]
    x59 = complex_enzyme_kivD_molecule_3me2oxo = x[45]
    x60 = complex_enzyme_kivD_molecule_isobutanal = x[46]
    x62 = molecule_isobutanol = x[47]
    x63 = complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal = x[48]
    x64 = complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol = x[49]
    x66 = complex_enzyme_atpase_metabolite_atp = x[50]
    x67 = complex_enzyme_atpase_metabolite_adp_metabolite_pi = x[51]


    return array([-3.0*x1**2*x42**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) - 15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 1.5*x1*x21**2*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 15.0*x1*x8*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) - 15.0*x1*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x5 + 1.5*x66,
 -15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 1.5*x5,
 -1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 15.0*x3*(10.0*x10 + x9)/(10.0*x3 + x8) + 15.0*x6 + 1.5*x9,
 -1.5*x12*x4*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) + 15.0*x14 - 1.5*x26*x4*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) - 15.0*x27*x4*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 1.5*x31 - 30.0*x38**2*x4**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) + 3.0*x43 + 15.0*x6 + 15.0*x67,
 15.0*x1*x2*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) - 36001.5*x5,
 1.5*x3*x4*(x5 + 10.0*x6)/(10.0*x1*x2 + x3*x4) + 36000.0*x5 - 15.0*x6,
 -15.0*x1*x8*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) + 15.0*x10 + 1.5*x13 - 1.5*x8*(10.0*x10 + x9)/(10.0*x3 + x8),
 15.0*x3*(10.0*x10 + x9)/(10.0*x3 + x8) - 36001.5*x9,
 -15.0*x10 + 1.5*x8*(10.0*x10 + x9)/(10.0*x3 + x8) + 36000.0*x9,
 -1.5*x12*x4*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) - 15.0*x12*(x17 + 10.0*x18)/(10.0*x12 + x16**2) + 15.0*x14 + 1.5*x17,
 15.0*x1*x8*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) - 36001.5*x13,
 1.5*x12*x4*(x13 + 10.0*x14)/(10.0*x1*x8 + x12*x4) + 36000.0*x13 - 15.0*x14,
 -30.0*x16**2*x20**2*x26*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 30.0*x16**2*x20**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) - 3.0*x16**2*(x17 + 10.0*x18)/(10.0*x12 + x16**2) + 30.0*x18 + 3.0*x23 + 3.0*x28,
 15.0*x12*(x17 + 10.0*x18)/(10.0*x12 + x16**2) - 36001.5*x17,
 1.5*x16**2*(x17 + 10.0*x18)/(10.0*x12 + x16**2) + 36000.0*x17 - 15.0*x18,
 -30.0*x16**2*x20**2*x26*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 30.0*x16**2*x20**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) - 1.5*x20*x50*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) - 1.5*x20*x62*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) + 3.0*x23 + 3.0*x28 + 15.0*x52 + 15.0*x64,
 -3.0*x1*x21**2*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 3.0*x21**2*x22**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) - 30.0*x21**2*(x35 + 10.0*x36)/(10.0*x21**2 + x34**2) + 30.0*x24 + 30.0*x32 + 3.0*x35,
 -3.0*x21**2*x22**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) - 3.0*x22**2*x27*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 15.0*x22*x46*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) - 15.0*x22*x58*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) + 30.0*x24 + 30.0*x29 + 1.5*x51 + 1.5*x63,
 15.0*x16**2*x20**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) - 36001.5*x23,
 1.5*x21**2*x22**2*(x23 + 10.0*x24)/(10.0*x16**2*x20**2 + x21**2*x22**2) + 36000.0*x23 - 15.0*x24,
 -15.0*x16**2*x20**2*x26*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 1.5*x26*x4*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) + 1.5*x28 + 15.0*x67,
 -1.5*x22**2*x27*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 15.0*x27*x4*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) + 15.0*x29 + 1.5*x31,
 15.0*x16**2*x20**2*x26*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) - 36001.5*x28,
 1.5*x22**2*x27*(x28 + 10.0*x29)/(10.0*x16**2*x20**2*x26 + x22**2*x27) + 36000.0*x28 - 15.0*x29,
 15.0*x27*x4*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) - 36001.5*x31,
 1.5*x1*x21**2*(x31 + 10.0*x32)/(x1*x21**2 + 10.0*x27*x4) + 36000.0*x31 - 15.0*x32,
 -30.0*x34**2*(x39 + 10.0*x40)/(10.0*x34**2 + x38**2) - 3.0*x34**2*(x35 + 10.0*x36)/(10.0*x21**2 + x34**2) + 30.0*x36 + 3.0*x39,
 15.0*x21**2*(x35 + 10.0*x36)/(10.0*x21**2 + x34**2) - 36001.5*x35,
 1.5*x34**2*(x35 + 10.0*x36)/(10.0*x21**2 + x34**2) + 36000.0*x35 - 15.0*x36,
 -30.0*x38**2*x4**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) - 3.0*x38**2*(x39 + 10.0*x40)/(10.0*x34**2 + x38**2) + 30.0*x40 + 3.0*x43,
 15.0*x34**2*(x39 + 10.0*x40)/(10.0*x34**2 + x38**2) - 36001.5*x39,
 1.5*x38**2*(x39 + 10.0*x40)/(10.0*x34**2 + x38**2) + 36000.0*x39 - 15.0*x40,
 -3.0*x1**2*x42**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) - 30.0*x42**2*(x47 + 10.0*x48)/(10.0*x42**2 + x46) + 30.0*x44 + 3.0*x47,
 15.0*x38**2*x4**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) - 36001.5*x43,
 1.5*x1**2*x42**2*(x43 + 10.0*x44)/(x1**2*x42**2 + 10.0*x38**2*x4**2) + 36000.0*x43 - 15.0*x44,
 -15.0*x22*x46*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) - 1.5*x46*(x47 + 10.0*x48)/(10.0*x42**2 + x46) + 15.0*x48 + 1.5*x51,
 15.0*x42**2*(x47 + 10.0*x48)/(10.0*x42**2 + x46) - 36001.5*x47,
 1.5*x46*(x47 + 10.0*x48)/(10.0*x42**2 + x46) + 36000.0*x47 - 15.0*x48,
 -1.5*x20*x50*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) - 15.0*x50*(x55 + 10.0*x56)/(10.0*x50 + x54) + 15.0*x52 + 1.5*x55,
 15.0*x22*x46*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) - 36001.5*x51,
 1.5*x20*x50*(x51 + 10.0*x52)/(x20*x50 + 10.0*x22*x46) + 36000.0*x51 - 15.0*x52,
 -15.0*x54*(x59 + 10.0*x60)/(10.0*x54 + x58) - 1.5*x54*(x55 + 10.0*x56)/(10.0*x50 + x54) + 15.0*x56 + 1.5*x59,
 15.0*x50*(x55 + 10.0*x56)/(10.0*x50 + x54) - 36001.5*x55,
 1.5*x54*(x55 + 10.0*x56)/(10.0*x50 + x54) + 36000.0*x55 - 15.0*x56,
 -15.0*x22*x58*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) - 1.5*x58*(x59 + 10.0*x60)/(10.0*x54 + x58) + 15.0*x60 + 1.5*x63,
 15.0*x54*(x59 + 10.0*x60)/(10.0*x54 + x58) - 36001.5*x59,
 1.5*x58*(x59 + 10.0*x60)/(10.0*x54 + x58) + 36000.0*x59 - 15.0*x60,
 -1.5*x20*x62*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) + 15.0*x64,
 15.0*x22*x58*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) - 36001.5*x63,
 1.5*x20*x62*(x63 + 10.0*x64)/(x20*x62 + 10.0*x22*x58) + 36000.0*x63 - 15.0*x64,
 15.0*x1*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) - 3.5*x66,
 1.5*x26*x4*(x66 + 10.0*x67)/(10.0*x1 + x26*x4) + 2.0*x66 - 15.0*x67])

  time = linspace(t0, tend, numPoints)
  #yinit= array([0.15, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0])
  #yinit = x_init
  yinit = x_init
  y = odeint(ode_fun, yinit, time)

  return time, y


time, result = simulateModel(0, 75, 1000)

p4 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 1: Remove All Complexes',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p4.line(time, result[:,0],line_width = 2, legend_label = 'atp', color = colors[3])
p4.line(time, result[:,1], line_width = 2,legend_label = 'glucose', color = colors[0])
p4.line(time, result[:,47], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p4.legend.click_policy = 'hide'
bokeh.io.show(p4)




/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in double_scalars
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykerne

RuntimeError: The size of the array returned by func (52) does not match the size of y0 (53).

## Entire process removing only one complex (near end)

In [ ]:
remove_list[3]

In [ ]:
%run prep_for_autoreduce.py
curr_list = [63]
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

In [ ]:
aa.f

In [ ]:

# Define correct initial conditions
spec_inds =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
              28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67] # indices of stuff we decided to keep
# Setup initial conditions

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 5, 7,9, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(spec_inds):
    if item in enz_list:
        x_init[i] = 0.15
    elif item in mol_list:
        x_init[i] = 30
    else:
        x_init[i] = 0
        




In [ ]:
x_init[64]


In [ ]:
from numpy import *
from matplotlib.pylab import *
from matplotlib.pyplot import *
from scipy.integrate import odeint 

def simulateModel(t0, tend, numPoints):
  
  #compartments
  default = 1e-06

  
  #boundary species
  
  def ode_fun(x, t): # REMOVE 
    x0 = enzyme_hex = x[0]
    x1 = metabolite_atp = x[1]
    x2 = molecule_glucose = x[2]
    x3 = molecule_g6p = x[3]
    x4 = metabolite_adp = x[4]
    x5 = complex_enzyme_hex_metabolite_atp_molecule_glucose = x[5]
    x6 = complex_enzyme_hex_metabolite_adp_molecule_g6p = x[6]
    x7 = enzyme_pgi = x[7]
    x8 = molecule_f6p = x[8]
    x9 = complex_enzyme_pgi_molecule_g6p = x[9]
    x10 = complex_enzyme_pgi_molecule_f6p = x[10]
    x11 = enzyme_pfk = x[11]
    x12 = molecule_f16p = x[12]
    x13 = complex_enzyme_pfk_metabolite_atp_molecule_f6p = x[13]
    x14 = complex_enzyme_pfk_metabolite_adp_molecule_f16p = x[14]
    x15= enzyme_ald_tpi = x[15]
    x16 = molecule_g3p = x[16]
    x17 = complex_enzyme_ald_tpi_molecule_f16p = x[17]
    x18 = complex_enzyme_ald_tpi_2x_molecule_g3p = x[18]
    x19 = enzyme_gapN = x[19]
    x20 = metabolite_nadp = x[20]
    x21 = molecule_3pg = x[21]
    x22= metabolite_nadph = x[22]
    x23 = complex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p = x[23]
    x24 = complex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg = x[24]
    x25 = enzyme_mGapDH = x[25]
    x26 = metabolite_pi = x[26]
    x27 = molecule_13bpg = x[27]
    x28 = complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p = x[28]
    x29 = complex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg = x[29]
    x30 = enzyme_pgk = x[30]
    x31 = complex_enzyme_pgk_metabolite_adp_molecule_13bpg = x[31]
    x32 = complex_enzyme_pgk_metabolite_atp_2x_molecule_3pg = x[32]
    x33 = enzyme_pgm = x[33]
    x34 = molecule_2pg = x[34]
    x35 = complex_enzyme_pgm_2x_molecule_3pg = x[35]
    x36 = complex_enzyme_pgm_2x_molecule_2pg = x[36]
    x37 = enzyme_eno = x[37]
    x38 = molecule_pep = x[38]
    x39 = complex_enzyme_eno_2x_molecule_2pg = x[39]
    x40 = complex_enzyme_eno_2x_molecule_pep = x[40]
    x41 = enzyme_pyk = x[41]
    x42 = molecule_pyruvate = x[42]
    x43 = complex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep = x[43]
    x44 = complex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate = x[44]
    x45 = enzyme_alsS = x[45]
    x46 = molecule_acetolac = x[46]
    x47 = complex_enzyme_alsS_2x_molecule_pyruvate = x[47]
    x48 = complex_enzyme_alsS_molecule_acetolac = x[48]
    x49 = enzyme_IlvC = x[49]
    x50 = molecule_23dih3mebut = x[50]
    x51 = complex_enzyme_IlvC_metabolite_nadph_molecule_acetolac = x[51]
    x52 = complex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut = x[52]
    x53 = enzyme_IlvD = x[53]
    x54 = molecule_3me2oxo = x[54]
    x55 = complex_enzyme_IlvD_molecule_23dih3mebut = x[55]
    x56= complex_enzyme_IlvD_molecule_3me2oxo = x[56]
    x57 = enzyme_kivD = x[57]
    x58 = molecule_isobutanal = x[58]
    x59 = complex_enzyme_kivD_molecule_3me2oxo = x[59]
    x60 = complex_enzyme_kivD_molecule_isobutanal = x[60]
    x61 = enzyme_yahk = x[61]
    x62 = molecule_isobutanol = x[62]
    x64 = complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol = x[63]
    x65 = enzyme_atpase = x[64]
    x66= complex_enzyme_atpase_metabolite_atp = x[65]
    x67 = complex_enzyme_atpase_metabolite_adp_metabolite_pi = x[66]


    return array([-15.0*x0*x1*x2 - 1.5*x0*x3*x4 + 1.5*x5 + 15.0*x6,
 -15.0*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 15.0*x1*x11*x8 - 1.5*x1*x21**2*x30 - 15.0*x1*x65 + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x5 + 1.5*x66,
 -15.0*x0*x1*x2 + 1.5*x5,
 -1.5*x0*x3*x4 - 15.0*x3*x7 + 15.0*x6 + 1.5*x9,
 -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 15.0*x27*x30*x4 + 1.5*x31 - 30.0*x38**2*x4**2*x41 + 3.0*x43 + 15.0*x6 + 15.0*x67,
 15.0*x0*x1*x2 - 36001.5*x5,
 1.5*x0*x3*x4 + 36000.0*x5 - 15.0*x6,
 15.0*x10 - 15.0*x3*x7 - 1.5*x7*x8 + 1.5*x9,
 -15.0*x1*x11*x8 + 15.0*x10 + 1.5*x13 - 1.5*x7*x8,
 15.0*x3*x7 - 36001.5*x9,
 -15.0*x10 + 1.5*x7*x8 + 36000.0*x9,
 -15.0*x1*x11*x8 - 1.5*x11*x12*x4 + 1.5*x13 + 15.0*x14,
 -1.5*x11*x12*x4 - 15.0*x12*x15 + 15.0*x14 + 1.5*x17,
 15.0*x1*x11*x8 - 36001.5*x13,
 1.5*x11*x12*x4 + 36000.0*x13 - 15.0*x14,
 -15.0*x12*x15 - 1.5*x15*x16**2 + 1.5*x17 + 15.0*x18,
 -3.0*x15*x16**2 - 30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 + 30.0*x18 + 3.0*x23 + 3.0*x28,
 15.0*x12*x15 - 36001.5*x17,
 1.5*x15*x16**2 + 36000.0*x17 - 15.0*x18,
 -15.0*x16**2*x19*x20**2 - 1.5*x19*x21**2*x22**2 + 1.5*x23 + 15.0*x24,
 -30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 - 1.5*x20*x49*x50 - 1.5*x20*x61*x62 + 3.0*x23 + 3.0*x28 + 15.0*x52 + 15.0*x64,
 -3.0*x1*x21**2*x30 - 3.0*x19*x21**2*x22**2 - 30.0*x21**2*x33 + 30.0*x24 + 30.0*x32 + 3.0*x35,
 -3.0*x19*x21**2*x22**2 - 3.0*x22**2*x25*x27 - 15.0*x22*x46*x49 - 14.9993750260406*x22*x58*x61 + 30.0*x24 + 30.0*x29 + 1.5*x51,
 15.0*x16**2*x19*x20**2 - 36001.5*x23,
 1.5*x19*x21**2*x22**2 + 36000.0*x23 - 15.0*x24,
 -15.0*x16**2*x20**2*x25*x26 - 1.5*x22**2*x25*x27 + 1.5*x28 + 15.0*x29,
 -15.0*x16**2*x20**2*x25*x26 - 1.5*x26*x4*x65 + 1.5*x28 + 15.0*x67,
 -1.5*x22**2*x25*x27 - 15.0*x27*x30*x4 + 15.0*x29 + 1.5*x31,
 15.0*x16**2*x20**2*x25*x26 - 36001.5*x28,
 1.5*x22**2*x25*x27 + 36000.0*x28 - 15.0*x29,
 -1.5*x1*x21**2*x30 - 15.0*x27*x30*x4 + 1.5*x31 + 15.0*x32,
 15.0*x27*x30*x4 - 36001.5*x31,
 1.5*x1*x21**2*x30 + 36000.0*x31 - 15.0*x32,
 -15.0*x21**2*x33 - 1.5*x33*x34**2 + 1.5*x35 + 15.0*x36,
 -3.0*x33*x34**2 - 30.0*x34**2*x37 + 30.0*x36 + 3.0*x39,
 15.0*x21**2*x33 - 36001.5*x35,
 1.5*x33*x34**2 + 36000.0*x35 - 15.0*x36,
 -15.0*x34**2*x37 - 1.5*x37*x38**2 + 1.5*x39 + 15.0*x40,
 -3.0*x37*x38**2 - 30.0*x38**2*x4**2*x41 + 30.0*x40 + 3.0*x43,
 15.0*x34**2*x37 - 36001.5*x39,
 1.5*x37*x38**2 + 36000.0*x39 - 15.0*x40,
 -1.5*x1**2*x41*x42**2 - 15.0*x38**2*x4**2*x41 + 1.5*x43 + 15.0*x44,
 -3.0*x1**2*x41*x42**2 - 30.0*x42**2*x45 + 30.0*x44 + 3.0*x47,
 15.0*x38**2*x4**2*x41 - 36001.5*x43,
 1.5*x1**2*x41*x42**2 + 36000.0*x43 - 15.0*x44,
 -15.0*x42**2*x45 - 1.5*x45*x46 + 1.5*x47 + 15.0*x48,
 -15.0*x22*x46*x49 - 1.5*x45*x46 + 15.0*x48 + 1.5*x51,
 15.0*x42**2*x45 - 36001.5*x47,
 1.5*x45*x46 + 36000.0*x47 - 15.0*x48,
 -1.5*x20*x49*x50 - 15.0*x22*x46*x49 + 1.5*x51 + 15.0*x52,
 -1.5*x20*x49*x50 - 15.0*x50*x53 + 15.0*x52 + 1.5*x55,
 15.0*x22*x46*x49 - 36001.5*x51,
 1.5*x20*x49*x50 + 36000.0*x51 - 15.0*x52,
 -15.0*x50*x53 - 1.5*x53*x54 + 1.5*x55 + 15.0*x56,
 -1.5*x53*x54 - 15.0*x54*x57 + 15.0*x56 + 1.5*x59,
 15.0*x50*x53 - 36001.5*x55,
 1.5*x53*x54 + 36000.0*x55 - 15.0*x56,
 -15.0*x54*x57 - 1.5*x57*x58 + 1.5*x59 + 15.0*x60,
 -14.9993750260406*x22*x58*x61 - 1.5*x57*x58 + 15.0*x60,
 15.0*x54*x57 - 36001.5*x59,
 1.5*x57*x58 + 36000.0*x59 - 15.0*x60,
 -1.5*x20*x61*x62 - 14.9993750260406*x22*x58*x61 + 15.0*x64,
 -1.5*x20*x61*x62 + 15.0*x64,
 1.5*x20*x61*x62 + 14.9993750260406*x22*x58*x61 - 15.0*x64,
 -15.0*x1*x65 - 1.5*x26*x4*x65 + 1.5*x66 + 15.0*x67,
 15.0*x1*x65 - 3.5*x66,
 1.5*x26*x4*x65 + 2.0*x66 - 15.0*x67])

  time = linspace(t0, tend, numPoints)
  #yinit= array([0.15, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0])
  #yinit = x_init
  yinit = x_init
  print(len(yinit))
  y = odeint(ode_fun, yinit, time)

  return time, y


time, result = simulateModel(0, 75, 1000)

p5 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 3: Remove 1 Complex (near end)',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p5.line(time, result[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p5.line(time, result[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p5.line(time, result[:,62], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p5.legend.click_policy = 'hide'
bokeh.io.show(p5)





## Try removing everything except atp, glucose, isobutanol

In [ ]:
%run prep_for_autoreduce.py
curr_list = remove_everything_list
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

In [ ]:
# Define correct initial conditions
spec_inds =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
              49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67]

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 7, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(x_init):
    if i in enz_list and i in spec_inds:
        x_init[i] = 0.15
        
    # atpase init cond
    if i == 65 and i in spec_inds:
        x_init[i] = 2.
        
    # set up other molecules
    if i in mol_list and i in spec_inds:
        x_init[i] = 30






In [ ]:
remove_everything_list

## Remove only complex 5

In [1]:
%run prep_for_autoreduce.py
curr_list = [5]
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

attempting to retain : [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/sympy/core/relational.py:470: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  deprecated_since_version="1.5"


Successful time-scale separation solution obtained.


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


236.17683197778143

In [ ]:
aa.f

In [ ]:

# Define correct initial conditions
spec_inds =  [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
              28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67] # indices of stuff we decided to keep
# Setup initial conditions

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 5, 7,9, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(spec_inds):
    if item in enz_list:
        x_init[i] = 0.15
    elif item in mol_list:
        x_init[i] = 30
    else:
        x_init[i] = 0
        





In [ ]:
from numpy import *
from matplotlib.pylab import *
from matplotlib.pyplot import *
from scipy.integrate import odeint 

def simulateModel(t0, tend, numPoints):
  
  #compartments
  default = 1e-06

  
  #boundary species
  
  def ode_fun(x, t): # REMOVE 
    x0 = enzyme_hex = x[0]
    x1 = metabolite_atp = x[1]
    x2 = molecule_glucose = x[2]
    x3 = molecule_g6p = x[3]
    x4 = metabolite_adp = x[4]
    #x5 = complex_enzyme_hex_metabolite_atp_molecule_glucose = x[5]
    x6 = complex_enzyme_hex_metabolite_adp_molecule_g6p = x[5]
    x7 = enzyme_pgi = x[6]
    x8 = molecule_f6p = x[7]
    x9 = complex_enzyme_pgi_molecule_g6p = x[8]
    x10 = complex_enzyme_pgi_molecule_f6p = x[9]
    x11 = enzyme_pfk = x[10]
    x12 = molecule_f16p = x[11]
    x13 = complex_enzyme_pfk_metabolite_atp_molecule_f6p = x[12]
    x14 = complex_enzyme_pfk_metabolite_adp_molecule_f16p = x[13]
    x15= enzyme_ald_tpi = x[14]
    x16 = molecule_g3p = x[15]
    x17 = complex_enzyme_ald_tpi_molecule_f16p = x[16]
    x18 = complex_enzyme_ald_tpi_2x_molecule_g3p = x[17]
    x19 = enzyme_gapN = x[18]
    x20 = metabolite_nadp = x[19]
    x21 = molecule_3pg = x[20]
    x22= metabolite_nadph = x[21]
    x23 = complex_enzyme_gapN_2x_metabolite_nadp_2x_molecule_g3p = x[22]
    x24 = complex_enzyme_gapN_2x_metabolite_nadph_2x_molecule_3pg = x[23]
    x25 = enzyme_mGapDH = x[24]
    x26 = metabolite_pi = x[25]
    x27 = molecule_13bpg = x[26]
    x28 = complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p = x[27]
    x29 = complex_enzyme_mGapDH_2x_metabolite_nadph_molecule_13bpg = x[28]
    x30 = enzyme_pgk = x[29]
    x31 = complex_enzyme_pgk_metabolite_adp_molecule_13bpg = x[30]
    x32 = complex_enzyme_pgk_metabolite_atp_2x_molecule_3pg = x[31]
    x33 = enzyme_pgm = x[32]
    x34 = molecule_2pg = x[33]
    x35 = complex_enzyme_pgm_2x_molecule_3pg = x[34]
    x36 = complex_enzyme_pgm_2x_molecule_2pg = x[35]
    x37 = enzyme_eno = x[36]
    x38 = molecule_pep = x[37]
    x39 = complex_enzyme_eno_2x_molecule_2pg = x[38]
    x40 = complex_enzyme_eno_2x_molecule_pep = x[39]
    x41 = enzyme_pyk = x[40]
    x42 = molecule_pyruvate = x[41]
    x43 = complex_enzyme_pyk_2x_metabolite_adp_2x_molecule_pep = x[42]
    x44 = complex_enzyme_pyk_2x_metabolite_atp_2x_molecule_pyruvate = x[43]
    x45 = enzyme_alsS = x[44]
    x46 = molecule_acetolac = x[45]
    x47 = complex_enzyme_alsS_2x_molecule_pyruvate = x[46]
    x48 = complex_enzyme_alsS_molecule_acetolac = x[47]
    x49 = enzyme_IlvC = x[48]
    x50 = molecule_23dih3mebut = x[49]
    x51 = complex_enzyme_IlvC_metabolite_nadph_molecule_acetolac = x[50]
    x52 = complex_enzyme_IlvC_metabolite_nadp_molecule_23dih3mebut = x[51]
    x53 = enzyme_IlvD = x[52]
    x54 = molecule_3me2oxo = x[53]
    x55 = complex_enzyme_IlvD_molecule_23dih3mebut = x[54]
    x56= complex_enzyme_IlvD_molecule_3me2oxo = x[55]
    x57 = enzyme_kivD = x[56]
    x58 = molecule_isobutanal = x[57]
    x59 = complex_enzyme_kivD_molecule_3me2oxo = x[58]
    x60 = complex_enzyme_kivD_molecule_isobutanal = x[59]
    x61 = enzyme_yahk = x[60]
    x62 = molecule_isobutanol = x[61]
    x63 = x[62]
    x64 = complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol = x[63]
    x65 = enzyme_atpase = x[64]
    x66= complex_enzyme_atpase_metabolite_atp = x[65]
    x67 = complex_enzyme_atpase_metabolite_adp_metabolite_pi = x[66]


    return array([-14.9993750260406*x0*x1*x2 - 1.5*x0*x3*x4 + 15.0*x6,
 -14.9993750260406*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 15.0*x1*x11*x8 - 1.5*x1*x21**2*x30 - 15.0*x1*x65 + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x66,
 -14.9993750260406*x0*x1*x2,
 -1.5*x0*x3*x4 - 15.0*x3*x7 + 15.0*x6 + 1.5*x9,
 -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 15.0*x27*x30*x4 + 1.5*x31 - 30.0*x38**2*x4**2*x41 + 3.0*x43 + 15.0*x6 + 15.0*x67,
 14.9993750260406*x0*x1*x2 + 1.5*x0*x3*x4 - 15.0*x6,
 15.0*x10 - 15.0*x3*x7 - 1.5*x7*x8 + 1.5*x9,
 -15.0*x1*x11*x8 + 15.0*x10 + 1.5*x13 - 1.5*x7*x8,
 15.0*x3*x7 - 36001.5*x9,
 -15.0*x10 + 1.5*x7*x8 + 36000.0*x9,
 -15.0*x1*x11*x8 - 1.5*x11*x12*x4 + 1.5*x13 + 15.0*x14,
 -1.5*x11*x12*x4 - 15.0*x12*x15 + 15.0*x14 + 1.5*x17,
 15.0*x1*x11*x8 - 36001.5*x13,
 1.5*x11*x12*x4 + 36000.0*x13 - 15.0*x14,
 -15.0*x12*x15 - 1.5*x15*x16**2 + 1.5*x17 + 15.0*x18,
 -3.0*x15*x16**2 - 30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 + 30.0*x18 + 3.0*x23 + 3.0*x28,
 15.0*x12*x15 - 36001.5*x17,
 1.5*x15*x16**2 + 36000.0*x17 - 15.0*x18,
 -15.0*x16**2*x19*x20**2 - 1.5*x19*x21**2*x22**2 + 1.5*x23 + 15.0*x24,
 -30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 - 1.5*x20*x49*x50 - 1.5*x20*x61*x62 + 3.0*x23 + 3.0*x28 + 15.0*x52 + 15.0*x64,
 -3.0*x1*x21**2*x30 - 3.0*x19*x21**2*x22**2 - 30.0*x21**2*x33 + 30.0*x24 + 30.0*x32 + 3.0*x35,
 -3.0*x19*x21**2*x22**2 - 3.0*x22**2*x25*x27 - 15.0*x22*x46*x49 - 15.0*x22*x58*x61 + 30.0*x24 + 30.0*x29 + 1.5*x51 + 1.5*x63,
 15.0*x16**2*x19*x20**2 - 36001.5*x23,
 1.5*x19*x21**2*x22**2 + 36000.0*x23 - 15.0*x24,
 -15.0*x16**2*x20**2*x25*x26 - 1.5*x22**2*x25*x27 + 1.5*x28 + 15.0*x29,
 -15.0*x16**2*x20**2*x25*x26 - 1.5*x26*x4*x65 + 1.5*x28 + 15.0*x67,
 -1.5*x22**2*x25*x27 - 15.0*x27*x30*x4 + 15.0*x29 + 1.5*x31,
 15.0*x16**2*x20**2*x25*x26 - 36001.5*x28,
 1.5*x22**2*x25*x27 + 36000.0*x28 - 15.0*x29,
 -1.5*x1*x21**2*x30 - 15.0*x27*x30*x4 + 1.5*x31 + 15.0*x32,
 15.0*x27*x30*x4 - 36001.5*x31,
 1.5*x1*x21**2*x30 + 36000.0*x31 - 15.0*x32,
 -15.0*x21**2*x33 - 1.5*x33*x34**2 + 1.5*x35 + 15.0*x36,
 -3.0*x33*x34**2 - 30.0*x34**2*x37 + 30.0*x36 + 3.0*x39,
 15.0*x21**2*x33 - 36001.5*x35,
 1.5*x33*x34**2 + 36000.0*x35 - 15.0*x36,
 -15.0*x34**2*x37 - 1.5*x37*x38**2 + 1.5*x39 + 15.0*x40,
 -3.0*x37*x38**2 - 30.0*x38**2*x4**2*x41 + 30.0*x40 + 3.0*x43,
 15.0*x34**2*x37 - 36001.5*x39,
 1.5*x37*x38**2 + 36000.0*x39 - 15.0*x40,
 -1.5*x1**2*x41*x42**2 - 15.0*x38**2*x4**2*x41 + 1.5*x43 + 15.0*x44,
 -3.0*x1**2*x41*x42**2 - 30.0*x42**2*x45 + 30.0*x44 + 3.0*x47,
 15.0*x38**2*x4**2*x41 - 36001.5*x43,
 1.5*x1**2*x41*x42**2 + 36000.0*x43 - 15.0*x44,
 -15.0*x42**2*x45 - 1.5*x45*x46 + 1.5*x47 + 15.0*x48,
 -15.0*x22*x46*x49 - 1.5*x45*x46 + 15.0*x48 + 1.5*x51,
 15.0*x42**2*x45 - 36001.5*x47,
 1.5*x45*x46 + 36000.0*x47 - 15.0*x48,
 -1.5*x20*x49*x50 - 15.0*x22*x46*x49 + 1.5*x51 + 15.0*x52,
 -1.5*x20*x49*x50 - 15.0*x50*x53 + 15.0*x52 + 1.5*x55,
 15.0*x22*x46*x49 - 36001.5*x51,
 1.5*x20*x49*x50 + 36000.0*x51 - 15.0*x52,
 -15.0*x50*x53 - 1.5*x53*x54 + 1.5*x55 + 15.0*x56,
 -1.5*x53*x54 - 15.0*x54*x57 + 15.0*x56 + 1.5*x59,
 15.0*x50*x53 - 36001.5*x55,
 1.5*x53*x54 + 36000.0*x55 - 15.0*x56,
 -15.0*x54*x57 - 1.5*x57*x58 + 1.5*x59 + 15.0*x60,
 -15.0*x22*x58*x61 - 1.5*x57*x58 + 15.0*x60 + 1.5*x63,
 15.0*x54*x57 - 36001.5*x59,
 1.5*x57*x58 + 36000.0*x59 - 15.0*x60,
 -1.5*x20*x61*x62 - 15.0*x22*x58*x61 + 1.5*x63 + 15.0*x64,
 -1.5*x20*x61*x62 + 15.0*x64,
 15.0*x22*x58*x61 - 36001.5*x63,
 1.5*x20*x61*x62 + 36000.0*x63 - 15.0*x64,
 -15.0*x1*x65 - 1.5*x26*x4*x65 + 1.5*x66 + 15.0*x67,
 15.0*x1*x65 - 3.5*x66,
 1.5*x26*x4*x65 + 2.0*x66 - 15.0*x67])

  time = linspace(t0, tend, numPoints)
  #yinit= array([0.15, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.0, 0.15, 30.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0])
  #yinit = x_init
  yinit = x_init
  print(len(yinit))
  y = odeint(ode_fun, yinit, time)

  return time, y


time, result = simulateModel(0, 750, 1000)

p5 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 3: Remove 1 Complex (near end)',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p5.line(time, result[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p5.line(time, result[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p5.line(time, result[:,62], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p5.legend.click_policy = 'hide'
bokeh.io.show(p5)






# Try only retain glucose, isobutanol, atp, enzymes

In [2]:
remove_everything_list2 = [3,
# 4, # adp
6,
8,
 10,
12,
13,
 14,
 16,
17,
18,
#20, # nadp
 21,
#22, # nadph
23,
24,
#26, # pi
27,
 28,
 29,
 31,
 32,
 34,
 35,
 36,
 38,
 39,
 40,
 42,
 43,
 44,
 46,
 47,
 48,
 50,
 51,
 52,
 54,
 55,
 56,
 58,
 59,
 60,
 63,
 64,
 66,
 67]

In [3]:
%run prep_for_autoreduce.py
curr_list = remove_everything_list2
timescale_list = []
for i in curr_list:
    #/print(i)
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)   
        
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm) 

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
sys_reduce.get_error_metric(aa)

attempting to retain : [0, 1, 2, 4, 5, 7, 9, 11, 15, 19, 20, 22, 25, 26, 30, 33, 37, 41, 45, 49, 53, 57, 61, 62, 65]
The time-scale separation that retains states [0, 1, 2, 4, 5, 7, 9, 11, 15, 19, 20, 22, 25, 26, 30, 33, 37, 41, 45, 49, 53, 57, 61, 62, 65] does not work because the state-variables [x21, x34, x10, x16, x60, x42, x66, x63, x14, x38, x67, x29, x64] appear in the reduced model


/Users/ankitaroychoudhury/autoReduce/auto_reduce/model_reduction.py:368: UserWarning: Check model consistency
  warnings.warn('Check model consistency')


AttributeError: 'NoneType' object has no attribute 'x'

In [7]:
type(bb)

NoneType

In [ ]:

# Define correct initial conditions
spec_inds =  [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
              28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67] # indices of stuff we decided to keep
# Setup initial conditions

# want all enzymes = 0.15, glucose, atp, nadp, maybe pi = 30, atpase_init = 2.0
x_init = np.zeros(len(spec_inds))


# set up init for all enzymes
enz_list = [0, 5, 7,9, 11, 15, 19, 25, 30, 33, 37, 41, 45, 49, 53, 57, 61, 65]
# indices for gluc, atp, nadp, pi
mol_list = [2, 1, 20, 26]

for i,item in enumerate(spec_inds):
    if item in enz_list:
        x_init[i] = 0.15
    elif item in mol_list:
        x_init[i] = 30
    else:
        x_init[i] = 0
        




